# Welford's Method for Computing Variance

[Welford’s method for computing variance](https://jonisalonen.com/2013/deriving-welfords-method-for-computing-variance/ shows single-pass method for computing the variance. This notebook shows computing method variance using batch-separated datasets.

In [1]:
import numpy as np

In [2]:
seed = 42

generator = np.random.default_rng(seed=seed)
samples = generator.normal(loc=5, scale=3, size=10000)

print(f'{len(samples) =}')
print(f'{samples.mean() =}')
print(f'{samples.std() =}')

len(samples) =10000
samples.mean() =4.969250373757966
samples.std() =3.018857305611123


## One-pass Variance Computation

Given a sample $x_1, ..., x_N$, the variance of the sample is defined as:
$$
s^2 = \frac{\sum_{i=1}^{N}(x_i-\bar{x})^2}{N-1}
$$
Here $\bar{x}$ is the mean of the sample: $\bar{x} = \frac{1}{N}\sum_{i=1}^{N}x_i $.

$$
\begin{eqnarray}
s^2 &=& \frac{\sum_{i=1}^{N}(x_i-\bar{x})^2}{N-1} \\
&=& \frac{\sum_{i=1}^{N}(x_i^2-2\bar{x}x_i+N\bar{x}^2)}{N-1} \\
&=& \frac{\sum_{i=1}^{N}x_i^2-2N\bar{x}^2+N\bar{x}^2}{N-1} \\
&=& \frac{\sum_{i=1}^{N}x_i^2-N\bar{x}^2}{N-1}
\end{eqnarray}
$$

$$
\begin{eqnarray}
\because \bar{x} &=& \frac{1}{N}\sum_{i=1}^{N}x_i \\
\sum_{i=1}^{N}x_i &=& N\bar{x}
\end{eqnarray}
$$


```
variance(samples):
  sum := 0
  sumsq := 0
  for x in samples:
    sum := sum + x
    sumsq := sumsq + x**2
  mean := sum/N 
  return (sumsq - N*mean**2)/(N-1)
```

In [3]:
def onepass_variance(samples):
    sum = 0
    sumsq = 0
    N = len(samples)
    
    for x in samples:
        sum = sum + x
        sumsq = sumsq + x**2
    mean = sum / N
    var = (sumsq - N*mean**2) / (N-1)

    return var

In [4]:
var = onepass_variance(samples)
std = np.sqrt(var)

print(f'{std =}')

std =3.0190082597980794


## Single-pass Variance Computation

The sum of squared differences for $N$ and $N-1$ samples is:

$$
\begin{eqnarray}
\sum_{i=1}^N(x_i-\bar{x}_N)^2 - \sum_{i=1}^{N-1}(x_i-\bar{x}_{N-1})^2 &=& (x_N-\bar{x}_N)^2 + \sum_{i=1}^{N-1}((x_i-\bar{x}_N)^2-(x_i-\bar{x}_{N-1})^2) \\
&=& (x_N-\bar{x}_N)^2 + \sum_{i=1}^{N-1}(x_i^2-2x_i\bar{x}_N+\bar{x}_N^2-x_i^2+2x_i\bar{x}_{N-1}-\bar{x}_{N-1}^2) \\
&=& (x_N-\bar{x}_N)^2 + \sum_{i=1}^{N-1}(-2x_i\bar{x}_N+\bar{x}_N^2+2x_i\bar{x}_{N-1}-\bar{x}_{N-1}^2) \\
&=& (x_N-\bar{x}_N)^2 + \sum_{i=1}^{N-1}(\bar{x}_N^2-\bar{x}_{N-1}^2+2x_i\bar{x}_{N-1}-2x_i\bar{x}_N) \\
&=& (x_N-\bar{x}_N)^2 + \sum_{i=1}^{N-1}((\bar{x}_N+\bar{x}_{N-1})(\bar{x}_N-\bar{x}_{N-1})-2x_i(\bar{x}_N-\bar{x}_{N-1})) \\
&=& (x_N-\bar{x}_N)^2 + \sum_{i=1}^{N-1}((\bar{x}_N+\bar{x}_{N-1}-2x_i)(\bar{x}_N-\bar{x}_{N-1})) \\
&=& (x_N-\bar{x}_N)^2 + (\bar{x}_N-\bar{x}_{N-1})\sum_{i=1}^{N-1}(\bar{x}_N+\bar{x}_{N-1}-2x_i) \\
&=& (x_N-\bar{x}_N)^2 + (\bar{x}_N-\bar{x}_{N-1})((N-1)\bar{x}_N+(N-1)\bar{x}_{N-1}-2\sum_{i=1}^{N-1}x_i) \\
&=& (x_N-\bar{x}_N)^2 + (\bar{x}_N-\bar{x}_{N-1})((N-1)\frac{1}{N}\sum_{i=1}^{N}x_i+(N-1)\frac{1}{N-1}\sum_{i=1}^{N-1}x_i-2\sum_{i=1}^{N-1}x_i) \\
&=& (x_N-\bar{x}_N)^2 + (\bar{x}_N-\bar{x}_{N-1})(\sum_{i=1}^{N}x_i-\frac{1}{N}\sum_{i=1}^{N}x_i-\sum_{i=1}^{N-1}x_i) \\
&=& (x_N-\bar{x}_N)^2 + (\bar{x}_N-\bar{x}_{N-1})(x_N-\bar{x}_N) \qquad \because \sum_{i=1}^{N}x_i-\sum_{i=1}^{N-1}x_i=x_N \\
&=& (x_N-\bar{x}_N)(x_N-\bar{x}_N+\bar{x}_N-\bar{x}_{N-1}) \\
&=& (x_N-\bar{x}_N)(x_N-\bar{x}_{N-1})
\end{eqnarray}
$$


```
variance(samples):
  M := 0
  S := 0
  for k from 1 to N:
    x := samples[k]
    oldM := M
    M := M + (x-M)/k
    S := S + (x-M)*(x-oldM)
  return S/(N-1)
```

In [5]:
def singlepass_variance(samples):
    M = 0
    S = 0
    
    for k in range(len(samples)):
        x = samples[k]
        oldM = M
        M = M + (x-M) / (k+1)
        S = S + (x-M) * (x-oldM)
    
    var = S / (len(samples)-1)
    return var

In [6]:
var = singlepass_variance(samples)
std = np.sqrt(var)

print(f'{std =}')

std =3.019008259798068


## Single-pass Variance Computation for Batched-separated Dataset

In [7]:
class SinglePassVarianceComputation():
    def __init__(self):
        self.M = 0
        self.S = 0
        self.N = 0

    def __call__(self, batch):
        for k in range(len(batch)):
            x = batch[k]
            oldM = self.M
            self.N = self.N + 1
            self.M = self.M + (x-self.M) / self.N
            self.S = self.S + (x-self.M) * (x-oldM)

        var = self.S / (self.N-1)
        return var

In [8]:
batch_size = 100

spvc = SinglePassVarianceComputation()
for i in range(0, len(samples), batch_size):
    var = spvc(samples[i:i+batch_size])
std = np.sqrt(var)

print(f'{std =}')

std =3.019008259798068
